$\newcommand{\giraffe}{\texttt{GiRaFFE}}$
# $\giraffe$: Solving GRFFE equations at a higher Finite Differencing order

### NRPy+ Source Code for this module: [Tutorial-GiRaFFE_Higher_Order.py](../edit/Tutorial-GiRaFFE_Higher_Order.py), which is fully documented in the [previous NRPy+ tutorial module](Tutorial-GiRaFFE_Higher_Order.ipynb) on using NRPy+ to construct Maxwell's equations and initial data as SymPy expressions.

This module focuses on using the equations developed in the [last tutorial](Tutorial-GiRaFFE_Higher_Order.ipynb) to build an Einstein Toolkit (ETK) thorn to solve the GRFFE equations in Cartesian coordinates. This tutorial will focus on implementing the time evolution aspects; others can be contructed to set up specific initial data.

When interfaced properly with the ETK, this module will propagate the initial data for $\tilde{S}_i$, $A_i$, and $\sqrt{\gamma} \Phi$, defined in the last tutorial, forward in time by integrating the equations for $\partial_t \tilde{S}_i$, $\partial_t A_i$ and $\partial_t [\sqrt{\gamma} \Phi]$ subject to spatial boundary conditions. The time evolution itself is handled by the $\text{MoL}$ (Method of Lines) thorn in the $\text{CactusNumerical}$ arrangement, and the boundary conditions by the $\text{Boundary}$ thorn in the $\text{CactusBase}$ arrangement. 

Similar to the other ETK modules we have built, we will construct the WaveToyNRPy module in two steps.

1. Call on NRPy+ to convert the SymPy expressions for the evolution equations into one C-code kernel.
1. Write the C code and linkages to the Einstein Toolkit infrastructure (i.e., the .ccl files) to complete this Einstein Toolkit module.


In [1]:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop

#Step 0: Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")
par.set_parval_from_str("grid::GridFuncMemAccess","ETK")
par.set_parval_from_str("outputC::outCverbose",False) 

# Step 1: Set the finite differencing order to 4.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 4)

# Step 1c: Call the MaxwellCartesian_Evol() function from within the
#          Maxwell/MaxwellCartesian_Evol.py module.
import GiRaFFE_Higher_Order as gho
gho.GiRaFFE_Higher_Order()

# Step 2: Create the C code output kernel.
Prereqs_to_print = [\
                   lhrh(lhs=gri.gfaccess("out_gfs","uU0"),rhs=gho.uU[0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","uU1"),rhs=gho.uU[1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","uU2"),rhs=gho.uU[2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","alphau0"),rhs=gho.alphau0),\
                   lhrh(lhs=gri.gfaccess("out_gfs","alpsqrtgam"),rhs=gho.alpsqrtgam),\
                   lhrh(lhs=gri.gfaccess("out_gfs","AevolParen"),rhs=gho.AevolParen),\
                   lhrh(lhs=gri.gfaccess("out_gfs","PevolParenU0"),rhs=gho.PevolParenU[0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","PevolParenU1"),rhs=gho.PevolParenU[1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","PevolParenU2"),rhs=gho.PevolParenU[2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU00"),rhs=gho.gammaUU[0][0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU01"),rhs=gho.gammaUU[0][1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU02"),rhs=gho.gammaUU[0][2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU11"),rhs=gho.gammaUU[1][1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU12"),rhs=gho.gammaUU[1][2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammaUU22"),rhs=gho.gammaUU[2][1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","gammadet"),rhs=gho.gammadet),\
                   ]

Conservs_to_print = [\
                    lhrh(lhs=gri.gfaccess("out_gfs","Stilde_rhsD0"),rhs=gho.Stilde_rhsD[0]),\
                    lhrh(lhs=gri.gfaccess("out_gfs","Stilde_rhsD1"),rhs=gho.Stilde_rhsD[1]),\
                    lhrh(lhs=gri.gfaccess("out_gfs","Stilde_rhsD2"),rhs=gho.Stilde_rhsD[2]),\
                    lhrh(lhs=gri.gfaccess("out_gfs","A_rhsD0"),rhs=gho.A_rhsD[0]),\
                    lhrh(lhs=gri.gfaccess("out_gfs","A_rhsD1"),rhs=gho.A_rhsD[1]),\
                    lhrh(lhs=gri.gfaccess("out_gfs","A_rhsD2"),rhs=gho.A_rhsD[2]),\
                    lhrh(lhs=gri.gfaccess("out_gfs","psi6Phi_rhs"),rhs=gho.psi6Phi_rhs),\
                    ]

Prereqs_CKernel = fin.FD_outputC("returnstring",Prereqs_to_print)
Prereqs_CKernel = "const double u0 = u0GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\n" + Prereqs_CKernel
Conservs_CKernel = fin.FD_outputC("returnstring",Conservs_to_print)
Conservs_CKernel = "const double u0 = u0GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];\n" + Conservs_CKernel

Prereqs_looped = loop.loop(["i2","i1","i0"],["0","0","0"],\
                           ["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
                           ["1","1","1"],["#pragma omp parallel for","",""],"",\
                           Prereqs_CKernel.replace("time","cctk_time"))

Conservs_looped = loop.loop(["i2","i1","i0"],["cctk_nghostzones[2]","cctk_nghostzones[1]","cctk_nghostzones[0]"],\
                            ["cctk_lsh[2]-cctk_nghostzones[2]","cctk_lsh[1]-cctk_nghostzones[1]",\
                             "cctk_lsh[0]-cctk_nghostzones[0]"],\
                            ["1","1","1"],["#pragma omp parallel for","",""],"",\
                            Conservs_CKernel.replace("time","cctk_time"))

# Step 3: Create directories for the thorn if they don't exist.
!mkdir GiRaFFE_HO     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir GiRaFFE_HO/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.

# Step 4: Write the C code kernel to file.
with open("GiRaFFE_HO/src/Prereqs.h", "w") as file:
    file.write(str(Prereqs_looped))

with open("GiRaFFE_HO/src/Conservs.h", "w") as file:
    file.write(str(Conservs_looped))

# Step 5: Import the function to calculate u0 and write it to a file.
import u0_smallb_Poynting__Cartesian.u0_smallb_Poynting__Cartesian as u0etc
#u0etc.compute_u0_smallb_Poynting__Cartesian(gammaDD,betaU,alpha,ValenciavU,BU)

with open("GiRaFFE_HO/src/computeu0_Cfunction.h", "w") as file:
    file.write(u0etc.computeu0_Cfunction)


We will also need a routine to compute new Valencia 3-velocities at each timestep using a conservative-to-primitive solver. Since we need $v^i_{(n)}$ everywhere, this will require us to compute $B^i$ everywhere. However, $B^i = \epsilon^{ijk} \partial_j A_k$ requires derivatives of $A_i$, so getting $B^i$ will require some finesse. A chief aspect of this will require using lower-order finite differencing in the ghost zones. To that end, we will create .h files for each finite differencing order $\leq 10$, as well as upwinded- and downwinded-derivatives at 2nd order. These will let us compute the derivative at the outermost gridpoints.

In [2]:
#AD = ixp.register_gridfunctions_for_single_rank1("AUX","AD",DIM=3)
# Step 2: Import the four metric

# We already have a handy function to define the Levi-Civita symbol in WeylScalars
import WeylScal4NRPy.WeylScalars_Cartesian as weyl
LeviCivitaDDD = weyl.define_LeviCivitaSymbol_rank3()
LeviCivitaUUU = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            LCijk = LeviCivitaDDD[i][j][k]
            LeviCivitaDDD[i][j][k] = LCijk * sp.sqrt(gho.gammadet)
            LeviCivitaUUU[i][j][k] = LCijk / sp.sqrt(gho.gammadet)

AD_dD = ixp.declarerank2("AD_dD","nosym")
BU = ixp.register_gridfunctions_for_single_rank1("AUX","BU")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 10)
BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_10.h",[\
                                                         lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                         lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                         lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 8)
BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_8.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 6)
BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_6.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 4)
BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_4.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 2)
BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

AD_ddnD = ixp.declarerank2("AD_ddnD","nosym")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 0:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x0D.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])
AD_dupD = ixp.declarerank2("AD_dupD","nosym")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 0:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x0U.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 1:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x1D.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 1:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x1U.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])

for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 2:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x2D.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 2:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

BU_to_print = fin.FD_outputC("GiRaFFE_HO/src/B_from_A_2x2U.h",[\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                                        lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])])



Wrote to file "GiRaFFE_HO/src/B_from_A_10.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_8.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_6.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_4.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x0D.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x0U.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x1D.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x1U.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x2D.h"
Wrote to file "GiRaFFE_HO/src/B_from_A_2x2U.h"


### Step 2: Interfacing with the Einstein Toolkit

#### Step 2a: Constructing the Einstein Toolkit C-code calling functions that include the C code kernels.

Now that we have generated the C code kernel *GiRaFFE_RHSs.h* and the parameters file *NRPy_params.h*, we will need to write C code to make use of these files. To do this, we can simply follow the example within the [IDScalarWaveNRPy tutorial module](Tutorial-ETK_thorn-IDScalarWaveNRPy.ipynb). Functions defined by these files will be called by the Einstein Toolkit scheduler (specified in schedule.ccl below).

Also, here we will write the logic that determines which files are called where in order to calculate $B^i$. 
1. Take the primary finite differencing order $N$ from the param.ccl file. Fill in the interior points with the corresponding FD order. 
1. Then, at each face, at $0+\text{cctk_nghostzones[face]}-1$ and $\text{cctk_lsh[face]}-\text{cctk_nghostzones[face]}+1$, calculate B at order $N-2$
1. Continue moving outwards: at the points $0+\text{cctk_nghostzones[face]}-p$ and $\text{cctk_lsh[face]}-\text{cctk_nghostzones[face]}+p$, calculate B at order $N-2p$.
1. When $\text{cctk_nghostzones[face]}-p = 0$, use the upwinding derivatives 

In [22]:
%%writefile GiRaFFE_HO/src/GiRaFFE.c
#include <math.h>
#include <stdio.h>
#include <stdlib.h>

#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"
#include "Symmetry.h"

const int MAXFACE = -1;
const int NUL     = +0;
const int MINFACE = +1;


// Declare boundary condition FACE_UPDATE function,
// which fills in the ghost zones with successively
// lower order finite differencing
void AtoB(const int ORDER, 
          const CCTK_REAL *AD0GF,const CCTK_REAL *AD1GF,const CCTK_REAL *AD2GF,
          const CCTK_REAL *BU0GF,const CCTK_REAL *BU1GF,const CCTK_REAL *BU2GF,
          const int i0min, const int i0max, 
          const int i1min, const int i1max, 
          const int i2min, const int i2max, 
          const int FACEX0, const int FACEX1, const int FACEX2) {
  
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
    
  const CCTK_REAL invdx0 = 1.0 / (CCTK_DELTA_SPACE(0));
  const CCTK_REAL invdx1 = 1.0 / (CCTK_DELTA_SPACE(1));
  const CCTK_REAL invdx2 = 1.0 / (CCTK_DELTA_SPACE(2));
  const CCTK_REAL *gammaDD00GF = gxx;
  const CCTK_REAL *gammaDD01GF = gxy;
  const CCTK_REAL *gammaDD02GF = gxz;
  const CCTK_REAL *gammaDD11GF = gyy;
  const CCTK_REAL *gammaDD12GF = gyz;
  const CCTK_REAL *gammaDD22GF = gzz;

  if(ORDER==8) {
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_8.h"
    }
  } else if(ORDER==6) {
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_6.h"
    }
  } else if(ORDER==4) {
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_4.h"
    }
  } else if(ORDER==2) {
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_2.h"
    } 
  } else if(ORDER==0) {
    if(FACEX0==MAXFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x0D.h"
        }
    } else if(FACEX0==MINFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x0U.h"
        }
    } else if(FACEX1==MAXFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x1D.h"
        }
    } else if(FACEX1==MINFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x1U.h"
        }
    } else if(FACEX2==MAXFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x2D.h"
        }
    } else if(FACEX2==MINFACE) {
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x2U.h"
        }
    } else {
        printf("ERROR. FACEX parameters not set properly.\n");
        exit(1);
    }
  } else {
    printf("ERROR. ORDER = %d not supported!\n",ORDER);
    exit(1);
  }
}

void driver_A_to_B(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  const int *NG = cctk_nghostzones;
  const int *Nx = cctk_lsh;
  CCTK_INT ORDER = NG[0]*2;
  for(int ii=0;ii<cctk_lsh[2]*cctk_lsh[1]*cctk_lsh[0];ii++) {
      BU0[ii] = 1.0 / 0.0;
      BU1[ii] = 1.0 / 0.0;
      BU2[ii] = 1.0 / 0.0;
  }

  AtoB(ORDER, AD0,AD1,AD2,BU0,BU1,BU2, NG[0],Nx[0]-NG[0],NG[1],Nx[1]-NG[1],NG[2],Nx[2]-NG[2], NUL,NUL,NUL);
  int imin[3] = { NG[0], NG[1], NG[2] };
  int imax[3] = { Nx[0]-NG[0], Nx[1]-NG[1], Nx[2]-NG[2] };
  while(ORDER>0) {
      // After updating each face, adjust imin[] and imax[] 
      //   to reflect the newly-updated face extents.
      ORDER -= 2;
      AtoB(ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imin[0]-1,imin[0], imin[1],imax[1], imin[2],imax[2], MINFACE,NUL,NUL); 
      if(ORDER!=0) imin[0]--;
      AtoB(ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imax[0],imax[0]+1, imin[1],imax[1], imin[2],imax[2], MAXFACE,NUL,NUL); 
      if(ORDER!=0) imax[0]++;

      AtoB(ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imin[0],imax[0], imin[1]-1,imin[1], imin[2],imax[2], NUL,MINFACE,NUL); 
      if(ORDER!=0) imin[1]--;
      AtoB(ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imin[0],imax[0], imax[1],imax[1]+1, imin[2],imax[2], NUL,MAXFACE,NUL); 
      if(ORDER!=0) imax[1]++;

      AtoB(ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imin[0],imax[0], imin[1],imax[1], imin[2]-1,imin[2], NUL,NUL,MINFACE); 
      if(ORDER!=0) imin[2]--;
      AtoB(ORDER, AD0,AD1,AD2,BU0,BU1,BU2, imin[0],imax[0], imin[1],imax[1], imax[2],imax[2]+1, NUL,NUL,MAXFACE); 
      if(ORDER!=0) imax[2]++;
    }
}

void GiRaFFE_HO_calc_prereqs(const cGH* restrict const cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,
                             const CCTK_REAL invdx0,const CCTK_REAL invdx1,const CCTK_REAL invdx2,
                             const CCTK_REAL *alphaGF, const CCTK_REAL *betaU0GF, const CCTK_REAL *betaU1GF, const CCTK_REAL *betaU2GF,
                             const CCTK_REAL *gammaDD00GF,const CCTK_REAL *gammaDD01GF,const CCTK_REAL *gammaDD02GF,const CCTK_REAL *gammaDD11GF,const CCTK_REAL *gammaDD12GF,const CCTK_REAL *gammaDD22GF,
                             const CCTK_REAL *ValenciavU0GF, const CCTK_REAL *ValenciavU1GF, const CCTK_REAL *ValenciavU2GF, const CCTK_REAL *u0GF,
                             const CCTK_REAL *AD0GF,const CCTK_REAL *AD1GF,const CCTK_REAL *AD2GF,const CCTK_REAL *psi6PhiGF,
                             CCTK_REAL *gammaUU00GF,CCTK_REAL *gammaUU01GF,CCTK_REAL *gammaUU02GF,CCTK_REAL *gammaUU11GF,CCTK_REAL *gammaUU12GF,CCTK_REAL *gammaUU22GF,CCTK_REAL *gammadetGF,
                             CCTK_REAL *uU0GF, CCTK_REAL *uU1GF, CCTK_REAL *uU2GF,
                             CCTK_REAL *alphau0GF, CCTK_REAL *alpsqrtgamGF, CCTK_REAL *AevolParenGF,
                             CCTK_REAL *PevolParenU0GF,CCTK_REAL *PevolParenU1GF,CCTK_REAL *PevolParenU2GF) {

  DECLARE_CCTK_PARAMETERS;

#include "Prereqs.h" 

}

void GiRaFFE_HO_calc_rhs(const cGH* restrict const cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,
                         const CCTK_REAL invdx0,const CCTK_REAL invdx1,const CCTK_REAL invdx2,
                         const CCTK_REAL *alphaGF, const CCTK_REAL *betaU0GF, const CCTK_REAL *betaU1GF, const CCTK_REAL *betaU2GF,
                         const CCTK_REAL *gammaDD00GF,const CCTK_REAL *gammaDD01GF,const CCTK_REAL *gammaDD02GF,const CCTK_REAL *gammaDD11GF,const CCTK_REAL *gammaDD12GF,const CCTK_REAL *gammaDD22GF,
                         const CCTK_REAL *gammaUU00GF,const CCTK_REAL *gammaUU01GF,const CCTK_REAL *gammaUU02GF,const CCTK_REAL *gammaUU11GF,const CCTK_REAL *gammaUU12GF,const CCTK_REAL *gammaUU22GF,const CCTK_REAL *gammadetGF,
                         const CCTK_REAL *ValenciavU0GF, const CCTK_REAL *ValenciavU1GF, const CCTK_REAL *ValenciavU2GF, const CCTK_REAL *u0GF,
                         const CCTK_REAL *uU0GF, const CCTK_REAL *uU1GF, const CCTK_REAL *uU2GF, 
                         const CCTK_REAL *AD0GF,const CCTK_REAL *AD1GF,const CCTK_REAL *AD2GF,const CCTK_REAL *psi6PhiGF,
                         const CCTK_REAL *alphau0GF, const CCTK_REAL *alpsqrtgamGF, const CCTK_REAL *AevolParenGF,
                         const CCTK_REAL *PevolParenU0GF,const CCTK_REAL *PevolParenU1GF,const CCTK_REAL *PevolParenU2GF,
                         CCTK_REAL *Stilde_rhsD0GF, CCTK_REAL *Stilde_rhsD1GF, CCTK_REAL *Stilde_rhsD2GF,
                         CCTK_REAL *A_rhsD0GF, CCTK_REAL *A_rhsD1GF, CCTK_REAL *A_rhsD2GF, CCTK_REAL *psi6Phi_rhsGF) {
  DECLARE_CCTK_PARAMETERS;

#include "Conservs.h"

}

void calc_u0(CCTK_REAL *ValenciavU0GF,CCTK_REAL *ValenciavU1GF,CCTK_REAL *ValenciavU2GF,CCTK_REAL alpha,
             const CCTK_REAL gammaDD00,const CCTK_REAL gammaDD01,const CCTK_REAL gammaDD02,const CCTK_REAL gammaDD11,const CCTK_REAL gammaDD12,const CCTK_REAL gammaDD22,
             CCTK_REAL *u0GF,const CCTK_INT idx)
{
  DECLARE_CCTK_PARAMETERS;
  CCTK_REAL u0;
  CCTK_REAL ValenciavU0 = ValenciavU0GF[idx];
  CCTK_REAL ValenciavU1 = ValenciavU1GF[idx];
  CCTK_REAL ValenciavU2 = ValenciavU2GF[idx];

#include "computeu0_Cfunction.h"

  u0GF[idx] = u0;
  ValenciavU0GF[idx] = ValenciavU0;
  ValenciavU1GF[idx] = ValenciavU1;
  ValenciavU2GF[idx] = ValenciavU2;
}

void GiRaFFE_HO_set_prereqs(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  const CCTK_REAL invdx0 = 1.0 / (CCTK_DELTA_SPACE(0));
  const CCTK_REAL invdx1 = 1.0 / (CCTK_DELTA_SPACE(1));
  const CCTK_REAL invdx2 = 1.0 / (CCTK_DELTA_SPACE(2));
  
#pragma omp parallel for
  for(int i2=0; i2<cctk_lsh[2]; i2++) {
      for(int i1=0; i1<cctk_lsh[1]; i1++) {
          for(int i0=0; i0<cctk_lsh[0]; i0++) {
              const CCTK_INT idx = CCTK_GFINDEX3D(cctkGH, i0,i1,i2);
              calc_u0(ValenciavU0,ValenciavU1,ValenciavU2,alp[idx],
                      gxx[idx],gxy[idx],gxz[idx],gyy[idx],gyz[idx],gzz[idx],
                      u0,idx);
          }
      }
  }

  GiRaFFE_HO_calc_prereqs(cctkGH,cctk_lsh,cctk_nghostzones,
                          invdx0, invdx1, invdx2,
                          alp, betax, betay, betaz,
                          gxx, gxy, gxz, gyy, gyz, gzz,
                          gammaUU00,gammaUU01,gammaUU02,gammaUU11,gammaUU12,gammaUU22,gammadet,
                          ValenciavU0, ValenciavU1, ValenciavU2, u0,
                          AD0, AD1, AD2, psi6Phi,
                          uU0, uU1, uU2,
                          alphau0, alpsqrtgam, AevolParen,
                          PevolParenU0, PevolParenU1, PevolParenU2);
}

void GiRaFFE_HO_set_rhs(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  const CCTK_REAL invdx0 = 1.0 / (CCTK_DELTA_SPACE(0));
  const CCTK_REAL invdx1 = 1.0 / (CCTK_DELTA_SPACE(1));
  const CCTK_REAL invdx2 = 1.0 / (CCTK_DELTA_SPACE(2));
  
  GiRaFFE_HO_calc_rhs(cctkGH,cctk_lsh,cctk_nghostzones,
                      invdx0, invdx1, invdx2,
                      alp, betax, betay, betaz,
                      gxx, gxy, gxz, gyy, gyz, gzz,
                      gammaUU00,gammaUU01,gammaUU02,gammaUU11,gammaUU12,gammaUU22,gammadet,
                      ValenciavU0, ValenciavU1, ValenciavU2, u0,
                      uU0, uU1, uU2,
                      AD0, AD1, AD2, psi6Phi,
                      alphau0, alpsqrtgam, AevolParen,
                      PevolParenU0, PevolParenU1, PevolParenU2,
                      Stilde_rhsD0, Stilde_rhsD1, Stilde_rhsD2,
                      A_rhsD0, A_rhsD1, A_rhsD2, psi6Phi_rhs);
}

/* Boundary Condition code adapted from WaveToyC thorn in ETK, implementing built-in
 * ETK BC functionality
 */
void GiRaFFE_HO_SelectBCs(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  const char *bctype;


  bctype = NULL;
  if (CCTK_EQUALS(bound,"flat") || CCTK_EQUALS(bound,"static") ||
      CCTK_EQUALS(bound,"radiation") || CCTK_EQUALS(bound,"robin") ||
      CCTK_EQUALS(bound,"none"))
  {
    bctype = bound;
  }
  else if (CCTK_EQUALS(bound,"zero"))
  {
    bctype = "scalar";
  }

  /* Uses all default arguments, so invalid table handle -1 can be passed */
  /*TODO: Add correct gridfunctions!*/
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::Stilde_rhsD0", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::Stilde_rhsD1", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::Stilde_rhsD2", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::A_rhsD0", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::A_rhsD1", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::A_rhsD2", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
  if (bctype && Boundary_SelectVarForBC (cctkGH, CCTK_ALL_FACES, 1, -1,
                                         "GiRaFFE_HO::psi6Phi_rhs", bctype) < 0)
  {
    CCTK_WARN (0, "GiRaFFE_HO_Boundaries: Error selecting boundary condition");
  }
}

void GiRaFFE_HO_InitSymBound(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
      
  int sym[3];

  sym[0] = 1;
  sym[1] = 1;
  sym[2] = 1;

  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::uU0");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::uU1");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::uU2");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::alphau0");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::alpsqrtgam");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::AevolParen");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::PevolParenU0");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::PevolParenU1");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::PevolParenU2");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::PsitildeD0");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::PsitildeD1");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::PsitildeD2");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::AD0");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::AD1");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::AD2");
  SetCartSymVN(cctkGH, sym,"GiRaFFE_HO::psi6Phi");

  return;
}

void GiRaFFE_HO_RegisterVars(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  CCTK_INT ierr CCTK_ATTRIBUTE_UNUSED = 0;
  /* Register all the evolved grid functions with MoL */
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::PsitildeD0"),  CCTK_VarIndex("GiRaFFE_HO::Psitilde_rhsD0"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::PsitildeD1"),  CCTK_VarIndex("GiRaFFE_HO::Psitilde_rhsD1"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::PsitildeD2"),  CCTK_VarIndex("GiRaFFE_HO::Psitilde_rhsD2"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::AD0"),  CCTK_VarIndex("GiRaFFE_HO::A_rhsD0"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::AD1"),  CCTK_VarIndex("GiRaFFE_HO::A_rhsD1"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::AD2"),  CCTK_VarIndex("GiRaFFE_HO::A_rhsD2"));
  ierr += MoLRegisterEvolved(CCTK_VarIndex("GiRaFFE_HO::psi6Phi"),  CCTK_VarIndex("GiRaFFE_HO::psi6Phi_rhs"));
  /* Register all the evolved Array functions with MoL */
  return;
}

Overwriting GiRaFFE_HO/src/GiRaFFE.c


We will also need to use the conservative to primitive solver from the old version of $\giraffe$, included here for convenience.

In [23]:
%%writefile GiRaFFE_HO/src/driver_conserv_to_prims_FFE.C
/* We evolve forward in time a set of functions called the 
 * "conservative variables" (magnetic field and Poynting vector), 
 * and any time the conserv's are updated, we must recover the 
 * primitive variables (velocities), before reconstructing & evaluating 
 * the RHSs of the MHD equations again. 
 *
 * This file contains the routine for this algebraic calculation. 
 * The velocity is calculated with formula (85), arXiv:1310.3274v2
 * $v^i = 4 \pi \alpha \gamma^{ij} {\tilde S}_j \gamma{-1/2} B^{-2} - \beta^i$ 
 * The force-free condition: $B^2>E^2$ is checked before computing the velocity.
 * and after imposing the constraint ${\tilde B}^i {\tilde S}_i = 0$
 
 * The procedure is as described in arXiv:1310.3274v2: 
 * 1. ${\tilde S}_i ->{\tilde S}_i - ({\tilde S}_j {\tilde B}^j) {\tilde B}^i/{\tilde B}^2$
 * 2. $f = \sqrt{(1-\gamma_{max}^{-2}){\tilde B}^4/(16 \pi^2 \gamma {\tilde S}^2)}$ 
 * 3. ${\tilde S}_i -> {\tilde S}_i min(1,f)
 * 4. $v^i = 4 \pi \alpha \gamma^{ij} {\tilde S}_j \gamma{-1/2} B^{-2} - \beta^i$
 * 5. ${\tilde n}_i v^i = 0$
 *
 * All equations are from: http://arxiv.org/pdf/1310.3274.pdf (v2)
 * */

#include "cctk.h"
#include <iostream>
#include <iomanip>
#include <fstream>
#include <sys/time.h>
#include <cmath>
#include <ctime>
#include <cstdlib>
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"
#include "Symmetry.h"

#ifndef M_PI
#define M_PI 3.141592653589793238463
#endif

//#include "GiRaFFE_headers.h"
//#include "inlined_functions.C"
#include "convert_ADM_to_BSSN__enforce_detgtij_eq_1__and_compute_gtupij.C"

extern "C" void GiRaFFE_conserv_to_prims_FFE(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  // We use proper C++ here, for file I/O later.
  using namespace std;

  // Here we convert ADM variables (from ADMBase) to the BSSN-based variables expected by this routine.
  GiRaFFE_convert_ADM_to_BSSN__enforce_detgtij_eq_1__and_compute_gtupij(cctkGH,cctk_lsh,  gxx,gxy,gxz,gyy,gyz,gzz,alp,
                                                                        gtxx,gtxy,gtxz,gtyy,gtyz,gtzz,
                                                                        gtupxx,gtupxy,gtupxz,gtupyy,gtupyz,gtupzz,
                                                                        phi_bssn,psi_bssn,lapm1);
  const int imin=0,jmin=0,kmin=0;
  const int imax=cctk_lsh[0],jmax=cctk_lsh[1],kmax=cctk_lsh[2];
  
  const CCTK_REAL dz = CCTK_DELTA_SPACE(2);

  CCTK_REAL error_int_numer=0,error_int_denom=0;

  CCTK_INT num_vel_limits=0,num_vel_nulls_current_sheet=0;
#pragma omp parallel for reduction(+:error_int_numer,error_int_denom,num_vel_limits,num_vel_nulls_current_sheet) schedule(static)
  for(int k=kmin;k<kmax;k++)
    for(int j=jmin;j<jmax;j++)
      for(int i=imin;i<imax;i++) {
        const int index = CCTK_GFINDEX3D(cctkGH,i,j,k);
        
        const CCTK_REAL rL = r[index];
        if(rL>min_radius_inside_of_which_conserv_to_prims_FFE_and_FFE_evolution_is_DISABLED) {

          const CCTK_REAL psi_bssnL = psi_bssn[index];
          const CCTK_REAL psi2 = psi_bssnL*psi_bssnL;
          const CCTK_REAL psi4 = psi2*psi2;
          const CCTK_REAL psim4 = 1.0/psi4;
          const CCTK_REAL sqrtg = psi4*psi2; // Determinant of 3-metric

          // \gamma_{ij}, computed from \tilde{\gamma}_{ij}
          const CCTK_REAL gxxL = psi4*gtxx[index];
          const CCTK_REAL gxyL = psi4*gtxy[index];
          const CCTK_REAL gxzL = psi4*gtxz[index];
          const CCTK_REAL gyyL = psi4*gtyy[index];
          const CCTK_REAL gyzL = psi4*gtyz[index];
          const CCTK_REAL gzzL = psi4*gtzz[index];

          // \gamma^{ij} = psim4 * \tilde{\gamma}^{ij}
          const CCTK_REAL gupxxL = psim4*gtupxx[index];
          const CCTK_REAL gupxyL = psim4*gtupxy[index];
          const CCTK_REAL gupxzL = psim4*gtupxz[index];
          const CCTK_REAL gupyyL = psim4*gtupyy[index];
          const CCTK_REAL gupyzL = psim4*gtupyz[index];
          const CCTK_REAL gupzzL = psim4*gtupzz[index];

          // Read in magnetic field and momentum variables once from memory, since memory access is expensive:
          const CCTK_REAL BU0L = BU0[index];
          const CCTK_REAL BU1L = BU1[index];
          const CCTK_REAL BU2L = BU2[index];

          // End of page 7 on http://arxiv.org/pdf/1310.3274.pdf
          const CCTK_REAL BtildexL = BU0L*sqrtg;
          const CCTK_REAL BtildeyL = BU1L*sqrtg;
          const CCTK_REAL BtildezL = BU2L*sqrtg;

          const CCTK_REAL Btilde_xL = gxxL*BtildexL + gxyL*BtildeyL + gxzL*BtildezL;
          const CCTK_REAL Btilde_yL = gxyL*BtildexL + gyyL*BtildeyL + gyzL*BtildezL;
          const CCTK_REAL Btilde_zL = gxzL*BtildexL + gyzL*BtildeyL + gzzL*BtildezL;

          CCTK_REAL StildeD0L = StildeD0[index];
          CCTK_REAL StildeD1L = StildeD1[index];
          CCTK_REAL StildeD2L = StildeD2[index];

          const CCTK_REAL StildeD0_orig = StildeD0L;
          const CCTK_REAL StildeD1_orig = StildeD1L;
          const CCTK_REAL StildeD2_orig = StildeD2L;

          const CCTK_REAL alpL = alp[index];
          const CCTK_REAL fourpialpha = 4.0*M_PI*alpL;

          const CCTK_REAL betaxL = betax[index];
          const CCTK_REAL betayL = betay[index];
          const CCTK_REAL betazL = betaz[index];
        
          //* 1. Just below Eq 90: Enforce orthogonality of B^i & S^i, so that B^i S_i = 0
          //*    Correction ${\tilde S}_i ->{\tilde S}_i - ({\tilde S}_j {\tilde B}^j) {\tilde B}_i/{\tilde B}^2$
          //*    NOTICE THAT THE {\tilde B}_i IS LOWERED, AS IT SHOULD BE. THIS IS A TYPO IN PASCHALIDIS ET AL.

          // First compute Btilde^i Stilde_i:
          const CCTK_REAL BtildeiSt_i = StildeD0L*BtildexL + StildeD1L*BtildeyL + StildeD2L*BtildezL;

          // Then compute (Btilde)^2
          const CCTK_REAL Btilde2 = gxxL*BtildexL*BtildexL + gyyL*BtildeyL*BtildeyL + gzzL*BtildezL*BtildezL
            + 2.0*(gxyL*BtildexL*BtildeyL + gxzL*BtildexL*BtildezL + gyzL*BtildeyL*BtildezL);

#define APPLY_GRFFE_FIXES

          // Now apply constraint: Stilde_i = Stilde_i - (Btilde^i Stilde_i) / (Btilde)^2
#ifdef APPLY_GRFFE_FIXES
          StildeD0L -= BtildeiSt_i*Btilde_xL/Btilde2;
          StildeD1L -= BtildeiSt_i*Btilde_yL/Btilde2;
          StildeD2L -= BtildeiSt_i*Btilde_zL/Btilde2;
#endif
          // Now that tildeS_i has been fixed, let's compute tildeS^i:
          CCTK_REAL mhd_st_upx = gupxxL*StildeD0L + gupxyL*StildeD1L + gupxzL*StildeD2L;
          CCTK_REAL mhd_st_upy = gupxyL*StildeD0L + gupyyL*StildeD1L + gupyzL*StildeD2L;
          CCTK_REAL mhd_st_upz = gupxzL*StildeD0L + gupyzL*StildeD1L + gupzzL*StildeD2L;

          // Just below Eq. 86 in http://arxiv.org/pdf/1310.3274.pdf:
          CCTK_REAL St2 = StildeD0L*mhd_st_upx + StildeD1L*mhd_st_upy + StildeD2L*mhd_st_upz;

          //* 2. Eq. 92: Factor $f = \sqrt{(1-\gamma_{max}^{-2}){\tilde B}^4/(16 \pi^2 \gamma {\tilde S}^2)}$ 

#ifdef APPLY_GRFFE_FIXES
          const CCTK_REAL gmax = GAMMA_SPEED_LIMIT;
          if(St2 > (1.0 - 1.0/(gmax*gmax))*Btilde2*Btilde2/ (16.0*M_PI*M_PI*sqrtg*sqrtg)) {
            const CCTK_REAL fact = sqrt((1.0 - 1.0/(gmax*gmax))/St2)*Btilde2/(4.0*M_PI*sqrtg);

            //* 3. ${\tilde S}_i -> {\tilde S}_i min(1,f)
            StildeD0L *= MIN(1.0,fact);
            StildeD1L *= MIN(1.0,fact);
            StildeD2L *= MIN(1.0,fact);

            // Recompute S^i
            mhd_st_upx = gupxxL*StildeD0L + gupxyL*StildeD1L + gupxzL*StildeD2L;
            mhd_st_upy = gupxyL*StildeD0L + gupyyL*StildeD1L + gupyzL*StildeD2L;
            mhd_st_upz = gupxzL*StildeD0L + gupyzL*StildeD1L + gupzzL*StildeD2L;
            /*
            printf("%e %e %e | %e %e %e | %e %e %e | oldgamma: %e %e should be > %e vfix\n",x[index],y[index],z[index],
                   BU0L,BU1L,BU2L,
                   St2,(1.0 - 1.0/(gmax*gmax))*Btilde2*Btilde2/ (16.0*M_PI*M_PI*sqrtg*sqrtg),gmax,
                   sqrt(Btilde2 / (Btilde2 - 16*M_PI*M_PI*sqrtg*sqrtg * St2 / Btilde2) ) , Btilde2,16*M_PI*M_PI*sqrtg*sqrtg * St2 / Btilde2  );
            //exit(1);
            */
            // Recompute Stilde^2:
            St2 = StildeD0L*mhd_st_upx + StildeD1L*mhd_st_upy + StildeD2L*mhd_st_upz;

            if( St2 >= Btilde2*Btilde2/ (16.0*M_PI*M_PI*sqrtg*sqrtg) ) {
              printf("ERROR: Velocity cap fix wasn't effective; still have B^2 > E^2\n"); exit(1);
            }
            num_vel_limits++;
          }
#endif

          //* 4. Eq. 85: $v^i = 4 pi \alpha \gamma^{ij} {\tilde S}_j \gamma{-1/2} B^{-2} - \beta^i$: 

          // See, e.g., Eq 71 in http://arxiv.org/pdf/1310.3274.pdf
          // ... or end of page 7 on http://arxiv.org/pdf/1310.3274.pdf:
          const CCTK_REAL B2 = Btilde2/(sqrtg*sqrtg);
          /* 
             Eq. 75: 
             v^i = \alpha \gamma^{ij} S_j / \mathcal{B}^2 - \beta^i
             Eq. 7: \mathcal{B}^{\mu} = B^{\mu}/\sqrt{4 \pi}
             -> v^i = 4 \pi \alpha \gamma^{ij} S_j / B^2 - \beta^i
             Eq. 79: \tilde{S_i} = \sqrt{\gamma} S_i
             -> v^i = 4 \pi \alpha \gamma^{ij} \tilde{S}_j / (\sqrt{\gamma} B^2) - \beta^i
          */
          const CCTK_REAL ValenciavU0L = fourpialpha*mhd_st_upx/(sqrtg*B2) - betaxL;
          const CCTK_REAL ValenciavU1L = fourpialpha*mhd_st_upy/(sqrtg*B2) - betayL;
          /* ValenciavU2L not necessarily const! See below. */
          CCTK_REAL ValenciavU2L = fourpialpha*mhd_st_upz/(sqrtg*B2) - betazL;

          //* 5. Eq. 94: ${\tilde n}_i v^i = 0$ in the current sheet region
          //     n^i is defined as the normal from the current sheet, which lies in the 
          //     xy-plane (z=0). So n = (0,0,1) 
#ifdef APPLY_GRFFE_FIXES
          if(current_sheet_null_v) {
            CCTK_REAL zL = z[index];
            if (fabs(zL) <= (4.0 + 1.0e-2)*dz ) {
              //ValenciavU2L = 0.0;
              ValenciavU2L = - (ValenciavU0L*gxzL + ValenciavU1L*gyzL) / gzzL;
            
              // ValenciavU2L reset: TYPICALLY WOULD RESET CONSERVATIVES TO BE CONSISTENT. LET'S NOT DO THAT, TO AVOID MESSING UP B-FIELDS

              if(1==1) {
                CCTK_REAL PRIMS[MAXNUMVARS];
                int ww=0;
                PRIMS[ww] = ValenciavU0L;    ww++;
                PRIMS[ww] = ValenciavU1L;    ww++;
                PRIMS[ww] = ValenciavU2L;    ww++;
                PRIMS[ww] = BU0L;    ww++;
                PRIMS[ww] = BU1L;    ww++;
                PRIMS[ww] = BU2L;    ww++;

                CCTK_REAL METRIC[NUMVARS_FOR_METRIC],dummy=0;
                ww=0;
                // FIXME: NECESSARY?
                //psi_bssn[index] = exp(phi[index]);
                METRIC[ww] = phi_bssn[index];ww++;
                METRIC[ww] = dummy;          ww++; // Don't need to set psi.
                METRIC[ww] = gtxx[index];    ww++;
                METRIC[ww] = gtxy[index];    ww++;
                METRIC[ww] = gtxz[index];    ww++;
                METRIC[ww] = gtyy[index];    ww++;
                METRIC[ww] = gtyz[index];    ww++;
                METRIC[ww] = gtzz[index];    ww++;
                METRIC[ww] = lapm1[index];   ww++;
                METRIC[ww] = betax[index];   ww++;
                METRIC[ww] = betay[index];   ww++;
                METRIC[ww] = betaz[index];   ww++;
                METRIC[ww] = gtupxx[index];  ww++;
                METRIC[ww] = gtupyy[index];  ww++;
                METRIC[ww] = gtupzz[index];  ww++;
                METRIC[ww] = gtupxy[index];  ww++;
                METRIC[ww] = gtupxz[index];  ww++;
                METRIC[ww] = gtupyz[index];  ww++;

                CCTK_REAL CONSERVS[NUM_CONSERVS] = {0.0, 0.0, 0.0}; // 3 conservative variables: Stilde_x, Stilde_y, Stilde_z
                GiRaFFE_compute_conservatives(PRIMS,METRIC, CONSERVS);

                StildeD0L = CONSERVS[STILDEX];
                StildeD1L = CONSERVS[STILDEY];
                StildeD2L = CONSERVS[STILDEZ];
              }
              num_vel_nulls_current_sheet++;
            }
          }
#endif

          ValenciavU0[index] = ValenciavU0L;      
          ValenciavU1[index] = ValenciavU1L;      
          ValenciavU2[index] = ValenciavU2L;      

          //Now we compute the difference between original & new conservatives, for diagnostic purposes:
          error_int_numer += fabs(StildeD0L - StildeD0_orig) + fabs(StildeD1L - StildeD1_orig) + fabs(StildeD2L - StildeD2_orig);
          error_int_denom += fabs(StildeD0_orig) + fabs(StildeD1_orig) + fabs(StildeD2_orig);
          /*
            if(fabs(ValenciavU0_orig) > 1e-13 && fabs(ValenciavU0L-ValenciavU0_orig)/ValenciavU0_orig > 1e-2) printf("BAD ValenciavU0: %e %e | %e %e %e\n",ValenciavU0L,ValenciavU0_orig,x[index],y[index],z[index]);
            if(fabs(ValenciavU1_orig) > 1e-13 && fabs(ValenciavU1L-ValenciavU1_orig)/ValenciavU1_orig > 1e-2) printf("BAD ValenciavU1: %e %e | %e %e %e\n",ValenciavU1L,ValenciavU1_orig,x[index],y[index],z[index]);
            if(fabs(ValenciavU2_orig) > 1e-13 && fabs(ValenciavU2L-ValenciavU2_orig)/ValenciavU2_orig > 1e-2) printf("BAD ValenciavU2: %e %e | %e %e %e\n",ValenciavU2L,ValenciavU2_orig,x[index],y[index],z[index]);
            error_int_numer += fabs(ValenciavU0L - ValenciavU0_orig) + fabs(ValenciavU1L - ValenciavU1_orig) + fabs(ValenciavU2L - ValenciavU2_orig);
            error_int_denom += fabs(ValenciavU0_orig) + fabs(ValenciavU1_orig) + fabs(ValenciavU2_orig);
          */


          StildeD0[index] = StildeD0L;
          StildeD1[index] = StildeD1L;
          StildeD2[index] = StildeD2L;
        }
      }

  CCTK_VInfo(CCTK_THORNSTRING,"FFEC2P: Lev: %d NumPts= %d | Error: %.3e, ErrDenom: %.3e, v_limits: %d / %d = %.3e, v_nulls: %d / %d = %.3e",
             (int)GetRefinementLevel(cctkGH),
             cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2],
             error_int_numer/(error_int_denom+1e-300),error_int_denom,
             /**/       num_vel_limits,            cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2],
             (CCTK_REAL)num_vel_limits/((CCTK_REAL)cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2]),
             /**/       num_vel_nulls_current_sheet,            cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2],
             (CCTK_REAL)num_vel_nulls_current_sheet/((CCTK_REAL)cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2]));
}

Overwriting GiRaFFE_HO/src/driver_conserv_to_prims_FFE.C


The previous driver depends on the function included below to convert ADM quantities to their BSSN counterparts.

In [5]:
%%writefile GiRaFFE_HO/src/convert_ADM_to_BSSN__enforce_detgtij_eq_1__and_compute_gtupij.C
#include "cctk.h"
#include <cmath>
#include <cstdio>
#include <cstdlib>
#include "cctk_Parameters.h"

void GiRaFFE_convert_ADM_to_BSSN__enforce_detgtij_eq_1__and_compute_gtupij
(const cGH *cctkGH,const int *cctk_lsh,
 CCTK_REAL *gxx,CCTK_REAL *gxy,CCTK_REAL *gxz,CCTK_REAL *gyy,CCTK_REAL *gyz,CCTK_REAL *gzz,const CCTK_REAL *alp,
 CCTK_REAL *gtxx,CCTK_REAL *gtxy,CCTK_REAL *gtxz,CCTK_REAL *gtyy,CCTK_REAL *gtyz,CCTK_REAL *gtzz,
 CCTK_REAL *gtupxx,CCTK_REAL *gtupxy,CCTK_REAL *gtupxz,CCTK_REAL *gtupyy,CCTK_REAL *gtupyz,CCTK_REAL *gtupzz,
 CCTK_REAL *phi,CCTK_REAL *psi,CCTK_REAL *lapm1) {
  DECLARE_CCTK_PARAMETERS; 
#pragma omp parallel for
  for(int k=0;k<cctk_lsh[2];k++) for(int j=0;j<cctk_lsh[1];j++) for(int i=0;i<cctk_lsh[0];i++) {
        const int index=CCTK_GFINDEX3D(cctkGH,i,j,k);
        const CCTK_REAL gxx_physL=gxx[index];
        const CCTK_REAL gxy_physL=gxy[index];
        const CCTK_REAL gxz_physL=gxz[index];
        const CCTK_REAL gyy_physL=gyy[index];
        const CCTK_REAL gyz_physL=gyz[index];
        const CCTK_REAL gzz_physL=gzz[index];

        /**********************************************************************
         * Compute \tilde{\gamma_{ij}}, phi, and psi (BSSN) from g_{ij} (ADM) *
         **********************************************************************/
        const CCTK_REAL gijdet = fabs(  gxx_physL * gyy_physL * gzz_physL + gxy_physL * gyz_physL * gxz_physL + gxz_physL * gxy_physL * gyz_physL
                                      - gxz_physL * gyy_physL * gxz_physL - gxy_physL * gxy_physL * gzz_physL - gxx_physL * gyz_physL * gyz_physL);

        const CCTK_REAL phiL = (1.0/12.0) * log(gijdet);
        const CCTK_REAL psiL = exp(phiL);

        const CCTK_REAL Psim4 = 1.0/(psiL*psiL*psiL*psiL);
        CCTK_REAL gtxxL = gxx_physL*Psim4;
        CCTK_REAL gtxyL = gxy_physL*Psim4;
        CCTK_REAL gtxzL = gxz_physL*Psim4;
        CCTK_REAL gtyyL = gyy_physL*Psim4;
        CCTK_REAL gtyzL = gyz_physL*Psim4;
        CCTK_REAL gtzzL = gzz_physL*Psim4;
        
        /*********************************
         * Apply det gtij = 1 constraint *
         *********************************/
        const CCTK_REAL gtijdet = gtxxL * gtyyL * gtzzL + gtxyL * gtyzL * gtxzL + gtxzL * gtxyL * gtyzL - 
          gtxzL * gtyyL * gtxzL - gtxyL * gtxyL * gtzzL - gtxxL * gtyzL * gtyzL;

        const CCTK_REAL gtijdet_Fm1o3 = fabs(1.0/cbrt(gtijdet));

        gtxxL = gtxxL * gtijdet_Fm1o3;
        gtxyL = gtxyL * gtijdet_Fm1o3; 
        gtxzL = gtxzL * gtijdet_Fm1o3; 
        gtyyL = gtyyL * gtijdet_Fm1o3; 
        gtyzL = gtyzL * gtijdet_Fm1o3; 
        gtzzL = gtzzL * gtijdet_Fm1o3;

        if(gtijdet<0.0) { CCTK_VWarn(CCTK_WARN_ALERT,__LINE__, __FILE__, CCTK_THORNSTRING,
                                     "WARNING: det[3-metric]<0.0 at point  %d %d %d | cctk_lsh: %d %d %d. Hopefully this is occurring in gz's! gtij_phys = %.2e %.2e %.2e %.2e %.2e %.2e gtij_new = %.2e %.2e %.2e %.2e %.2e %.2e | gijdet = %.2e | gtijdet = %.2e",
				     i,j,k,cctk_lsh[0],cctk_lsh[1],cctk_lsh[2],gxx_physL,gxy_physL,gxz_physL,gyy_physL,gyz_physL,gzz_physL,gtxxL,gtxyL,gtxzL,gtyyL,gtyzL,gtzzL,-gijdet,gtijdet); }

        const CCTK_REAL Psi4 = psiL*psiL*psiL*psiL;
        /*****************************************
         * Set all the needed BSSN gridfunctions *
         *****************************************/
        phi[index] = phiL;
        psi[index] = psiL;

        lapm1[index] = alp[index] - 1.0;

        gtxx[index] = gtxxL;
        gtxy[index] = gtxyL;
        gtxz[index] = gtxzL;
        gtyy[index] = gtyyL;
        gtyz[index] = gtyzL;
        gtzz[index] = gtzzL;

        gxx[index] = gtxxL*Psi4;
        gxy[index] = gtxyL*Psi4;
        gxz[index] = gtxzL*Psi4;
        gyy[index] = gtyyL*Psi4;
        gyz[index] = gtyzL*Psi4;
        gzz[index] = gtzzL*Psi4;

        gtupxx[index] =   ( gtyyL * gtzzL - gtyzL * gtyzL );
        gtupxy[index] = - ( gtxyL * gtzzL - gtyzL * gtxzL );
        gtupxz[index] =   ( gtxyL * gtyzL - gtyyL * gtxzL );
        gtupyy[index] =   ( gtxxL * gtzzL - gtxzL * gtxzL );
        gtupyz[index] = - ( gtxxL * gtyzL - gtxyL * gtxzL );
        gtupzz[index] =   ( gtxxL * gtyyL - gtxyL * gtxyL );
      }
}

Overwriting GiRaFFE_HO/src/convert_ADM_to_BSSN__enforce_detgtij_eq_1__and_compute_gtupij.C


This thorn should also use the common GRMHD variables provided by HydroBase, to allow it to interact easily with other MHD thorns. To that end, we will need to read in the common MHD variables at the beginning of our evolution, and then write the new values at the end of our evolution.

In [18]:
%%writefile GiRaFFE_HO/src/GiRaFFE_HydroBase_conversion.c
#include <math.h>
#include <stdio.h>
#include <stdlib.h>

#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"
#include "Symmetry.h"

void HydroBase_to_GiRaFFE(CCTK_ARGUMENTS) {
  /* BUi         <- Bvec[i]
   * ADi         <- Avec[i]
   * ValenciavUi <- vel[i]
   */
  DECLARE_CCTK_PARAMETERS;
  DECLARE_CCTK_ARGUMENTS;

  CCTK_INT  idx3;
  CCTK_INT *idx4;
#pragma omp parallel for
  for(int i2=0; i2<cctk_lsh[2]; i2++) {
      for(int i1=0; i1<cctk_lsh[1]; i1++) {
          for(int i0=0; i0<cctk_lsh[0]; i0++) {
              idx3 = CCTK_GFINDEX3D(cctkGH, i0,i1,i2);
              idx4[0] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,0);
              idx4[1] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,1);
              idx4[2] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,2);
              BU0[idx3] = Bvec[idx4[0]];
              BU1[idx3] = Bvec[idx4[1]];
              BU2[idx3] = Bvec[idx4[2]];
              AD0[idx3] = Avec[idx4[0]];
              AD1[idx3] = Avec[idx4[1]];
              AD2[idx3] = Avec[idx4[2]];
              ValenciavU0[idx3] = vel[idx4[0]];
              ValenciavU1[idx3] = vel[idx4[1]];
              ValenciavU2[idx3] = vel[idx4[2]];
              // We don't set Phi, because it is always set to zero in GiRaFFE ID.
          }
      }
  }
}

void GiRaFFE_to_HydroBase(CCTK_ARGUMENTS) {
  /* Bvec[i] <- BUi
   * Avec[i] <- ADi
   * vel[i]  <- ValenciavUi
   */
  DECLARE_CCTK_PARAMETERS;
  DECLARE_CCTK_ARGUMENTS;
  
  CCTK_INT  idx3;
  CCTK_INT *idx4;
#pragma omp parallel for
  for(int i2=0; i2<cctk_lsh[2]; i2++) {
      for(int i1=0; i1<cctk_lsh[1]; i1++) {
          for(int i0=0; i0<cctk_lsh[0]; i0++) {
              idx3 = CCTK_GFINDEX3D(cctkGH, i0,i1,i2);
              idx4[0] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,0);
              idx4[1] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,1);
              idx4[2] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,2);
              Bvec[idx4[0]] = BU0[idx3];
              Bvec[idx4[1]] = BU1[idx3];
              Bvec[idx4[2]] = BU2[idx3];
              Avec[idx4[0]] = AD0[idx3];
              Avec[idx4[1]] = AD1[idx3];
              Avec[idx4[2]] = AD2[idx3];
              vel[idx4[0]] = ValenciavU0[idx3];
              vel[idx4[1]] = ValenciavU1[idx3];
              vel[idx4[2]] = ValenciavU2[idx3];
              // We don't set Phi, because it is always set to zero in GiRaFFE ID thorns.
          }
      }
  }
}

Overwriting GiRaFFE_HO/src/GiRaFFE_HydroBase_conversion.c


#### Step 2b: CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure

Writing a module ("thorn") within the Einstein Toolkit requires that three "ccl" files be constructed, all in the root directory of the thorn:

1. $\text{interface.ccl}$: defines the gridfunction groups needed, and provides keywords denoting what this thorn provides and what it should inherit from other thorns. This file governs the interaction between this thorn and others; more information can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 
With "implements", we give our thorn its unique name. By "inheriting" other thorns, we tell the Toolkit that we will rely on variables that exist within those functions. Then, we tell the toolkit that we want the gridfunctions $A_i$, $\tilde{S}_i$, and $\sqrt{\gamma}\Phi$ to be visible to other thorns by using the keyword "public". 

In [17]:
%%writefile GiRaFFE_HO/interface.ccl
implements: GiRaFFE_HO

inherits: admbase HydroBase Boundary grid

USES INCLUDE: loopcontrol.h
USES INCLUDE: Symmetry.h
USES INCLUDE: Boundary.h
    
CCTK_INT FUNCTION MoLRegisterEvolved(CCTK_INT IN EvolvedIndex, CCTK_INT IN RHSIndex)
USES FUNCTION MoLRegisterEvolved

CCTK_INT FUNCTION GetBoundarySpecification(CCTK_INT IN size, CCTK_INT OUT ARRAY nboundaryzones, CCTK_INT OUT ARRAY is_internal, CCTK_INT OUT ARRAY is_staggered, CCTK_INT OUT ARRAY shiftout)
USES FUNCTION GetBoundarySpecification

CCTK_INT FUNCTION SymmetryTableHandleForGrid(CCTK_POINTER_TO_CONST IN cctkGH)
USES FUNCTION SymmetryTableHandleForGrid

CCTK_INT FUNCTION Boundary_SelectGroupForBC(CCTK_POINTER_TO_CONST IN GH, CCTK_INT IN faces, CCTK_INT IN boundary_width, CCTK_INT IN table_handle, CCTK_STRING IN group_name, CCTK_STRING IN bc_name)
USES FUNCTION Boundary_SelectGroupForBC

CCTK_INT FUNCTION Boundary_SelectVarForBC(CCTK_POINTER_TO_CONST IN GH, CCTK_INT IN faces, CCTK_INT IN boundary_width, CCTK_INT IN table_handle, CCTK_STRING IN var_name, CCTK_STRING IN bc_name)
USES FUNCTION Boundary_SelectVarForBC

public:
cctk_real GiRaFFE_aux type = GF Timelevels=1 tags='tensortypealias="Scalar"'
{
  uU0,uU1,uU2,alphau0,alpsqrtgam,AevolParen,PevolParenU0,PevolParenU1,PevolParenU2,
  gammaUU00,gammaUU01,gammaUU02,gammaUU11,gammaUU12,gammaUU22,gammadet
} "The evolved scalar fields"

public:
cctk_real GiRaFFE_Bs type = GF Timelevels=1 tags='tensortypealias="Scalar"'
{
  BU0,BU1,BU2
} "The B field"

public:
cctk_real GiRaFFE_Vs type = GF Timelevels=1 tags='tensortypealias="Scalar"'
{
  ValenciavU0,ValenciavU1,ValenciavU2
} "The Valencia 3-velocity"

public:
cctk_real GiRaFFE_rhs type = GF Timelevels=3 tags='tensortypealias="Scalar"'
{
  Stilde_rhsD0,Stilde_rhsD1,Stilde_rhsD2,A_rhsD0,A_rhsD1,A_rhsD2,psi6Phi_rhs
} "The evolved scalar fields"

public:
cctk_real GiRaFFE_vars type = GF Timelevels=3 tags='tensortypealias="Scalar"'
{
  StildeD0,StildeD1,StildeD2,AD0,AD1,AD2,psi6Phi
} "The evolved scalar fields"

public:
cctk_real GiRaFFE_u0 type = SCALAR Timelevels=1 tags='tensortypealias="Scalar"'
{
  u0
} "The zeroth component of the four velocity"

#vvvvvvv BSSN-based quantities, computed from ADM quantities.v vvvvvvv#
cctk_real BSSN_quantities type = GF TAGS='prolongation="none" Checkpoint="no"'
{
   gtxx,gtxy,gtxz,gtyy,gtyz,gtzz,gtupxx,gtupxy,gtupxz,gtupyy,gtupyz,gtupzz,phi_bssn,psi_bssn,lapm1
} "BSSN quantities, computed from ADM quantities"
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^#


Overwriting GiRaFFE_HO/interface.ccl


2. $\text{param.ccl}$: specifies free parameters within the thorn, enabling them to be set at runtime. It is required to provide allowed ranges and default values for each parameter. More information on this file's syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3). A number of parameters are defined, and more parameters can be easily added in later versions. We also set the number of timelevels we will store in memory.

In [24]:
%%writefile GiRaFFE_HO/param.ccl
shares: MethodOfLines

USES CCTK_INT MoL_Num_Evolved_Vars
USES CCTK_INT MoL_Num_ArrayEvolved_Vars

restricted:
CCTK_INT GiRaFFE_HO_MaxNumEvolvedVars "Number of evolved variables used by this thorn" ACCUMULATOR-BASE=MethodofLines::MoL_Num_Evolved_Vars STEERABLE=RECOVER
{
  7:7 :: "Number of evolved variables used by this thorn"
} 7

restricted:
CCTK_INT GiRaFFE_HO_MaxNumArrayEvolvedVars "Number of Array evolved variables used by this thorn" ACCUMULATOR-BASE=MethodofLines::MoL_Num_ArrayEvolved_Vars STEERABLE=RECOVER
{
  0:0 :: "Number of Array evolved variables used by this thorn"
} 0

restricted:
KEYWORD bound "Type of boundary condition to use"
{
  "flat"      :: "Flat (von Neumann, n grad phi = 0) boundary condition"
  "static"    :: "Static (Dirichlet, dphi/dt=0) boundary condition"
  "radiation" :: "Radiation boundary condition"
  "robin"     :: "Robin (phi(r) = C/r) boundary condition"
  "zero"      :: "Zero (Dirichlet, phi=0) boundary condition"
  "none"      :: "Apply no boundary condition"
} "static"

restricted:
CCTK_INT timelevels "Number of active timelevels" STEERABLE=RECOVER
{
  0:3 :: ""
} 3

restricted:
CCTK_REAL xi "The damping factor for the psi6Phi evolution equation"
{
 *:* :: "The damping factor for the psi6Phi evolution equation"
} 1.0

# SPEED LIMIT: Set maximum relativistic gamma factor
# 
REAL GAMMA_SPEED_LIMIT "Maximum relativistic gamma factor. Note the default is much higher than IllinoisGRMHD. (GRFFE can handle higher Lorentz factors)"
{
 1:* :: "Positive > 1, though you'll likely have troubles far above 2000."
} 2000.0

REAL min_radius_inside_of_which_conserv_to_prims_FFE_and_FFE_evolution_is_DISABLED "As parameter suggests, this is the minimum radius inside of which the conservatives-to-primitives solver is disabled. In the Aligned Rotator test, this should be set equal to R_NS_aligned_rotator." STEERABLE=ALWAYS
{
  -1.  :: "disable the conservative-to-primitive solver modification"
  (0:* :: "any positive value"
}  -1.


Overwriting GiRaFFE_HO/param.ccl


3. $\text{schedule.ccl}$: allocates storage for gridfunctions, defines how the thorn's functions should be scheduled in a broader simulation, and specifies the regions of memory written to or read from gridfunctions. $\text{schedule.ccl}$'s official documentation may be found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4). 

We first assign storage for both scalar gridfunctions, and then specify the standardized ETK "scheduling bins" in which we want each of our thorn's functions to run.

In [9]:
%%writefile GiRaFFE_HO/schedule.ccl
STORAGE: GiRaFFE_rhs[timelevels]
STORAGE: GiRaFFE_vars[timelevels]
STORAGE: GiRaFFE_aux[timelevels]
STORAGE: GiRaFFE_Bs[timelevels]

# POSTPOSTINITIAL
schedule GROUP GiRaFFE_PostPostInitial at CCTK_POSTPOSTINITIAL before MoL_PostStep after HydroBase_Con2Prim
{
} "HydroBase_Con2Prim in CCTK_POSTPOSTINITIAL sets conserv to prim then outer boundaries (OBs, which are technically disabled). The post OB SYNCs actually reprolongate the conservative variables, making cons and prims INCONSISTENT. So here we redo the con2prim, avoiding the SYNC afterward, then copy the result to other timelevels"

schedule GiRaFFE_HO_InitSymBound at BASEGRID
{
  LANG: C
  OPTIONS: global
} "Schedule symmetries"

schedule driver_A_to_B as driver_A_to_B IN MoL_CalcRHS
{
  LANG: C
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: GiRaFFE_HO::AD0(Everywhere)
  READS: GiRaFFE_HO::AD1(Everywhere)
  READS: GiRaFFE_HO::AD2(Everywhere)
  WRITES: GiRaFFE_HO::BU0(Everywhere)
  WRITES: GiRaFFE_HO::BU1(Everywhere)
  WRITES: GiRaFFE_HO::BU2(Everywhere)
} "Calculates the B-field from the vector potential"

schedule GiRaFFE_HO_set_prereqs as GiRaFFE_HO_set_prereqs IN MoL_CalcRHS after driver_A_to_B
{
  LANG: C
  READS: admbase::alp(Everywhere)
  READS: admbase::betax(Everywhere)
  READS: admbase::betay(Everywhere)
  READS: admbase::betaz(Everywhere)
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: GiRaFFE_HO::ValenciavU0(Everywhere)
  READS: GiRaFFE_HO::ValenciavU1(Everywhere)
  READS: GiRaFFE_HO::ValenciavU2(Everywhere)
  READS: GiRaFFE_HO::AD0(Everywhere)
  READS: GiRaFFE_HO::AD1(Everywhere)
  READS: GiRaFFE_HO::AD2(Everywhere)
  READS: GiRaFFE_HO::psi6Phi(Everywhere)
  WRITES: GiRaFFE_HO::uU0(Everywhere)
  WRITES: GiRaFFE_HO::uU1(Everywhere)
  WRITES: GiRaFFE_HO::uU2(Everywhere)
  WRITES: GiRaFFE_HO::alphau0(Everywhere)
  WRITES: GiRaFFE_HO::alpsqrtgam(Everywhere)
  WRITES: GiRaFFE_HO::AevolParen(Everywhere)
  WRITES: GiRaFFE_HO::PevolParenU0(Everywhere)
  WRITES: GiRaFFE_HO::PevolParenU1(Everywhere)
  WRITES: GiRaFFE_HO::PevolParenU2(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU00(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU01(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU02(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU11(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU12(Everywhere)
  WRITES: GiRaFFE_HO::gammaUU22(Everywhere)
  WRITES: GiRaFFE_HO::gammadet(Everywhere)
  WRITES: GiRaFFE_HO::u0(Everywhere)
  SYNC: GiRaFFE_aux
} "Sets prerequisite quantities for the GiRaFFE right-hand sides"

schedule GiRaFFE_HO_set_rhs as GiRaFFE_HO_Evolution IN MoL_CalcRHS after GiRaFFE_HO_set_prereqs
{
  LANG: C
  READS: admbase::alp(Everywhere)
  READS: admbase::betax(Everywhere)
  READS: admbase::betay(Everywhere)
  READS: admbase::betaz(Everywhere)
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: GiRaFFE_HO::ValenciavU0(Everywhere)
  READS: GiRaFFE_HO::ValenciavU1(Everywhere)
  READS: GiRaFFE_HO::ValenciavU2(Everywhere)
  READS: GiRaFFE_HO::AD0(Everywhere)
  READS: GiRaFFE_HO::AD1(Everywhere)
  READS: GiRaFFE_HO::AD2(Everywhere)
  READS: GiRaFFE_HO::uU0(Everywhere)
  READS: GiRaFFE_HO::uU1(Everywhere)
  READS: GiRaFFE_HO::uU2(Everywhere)
  READS: GiRaFFE_HO::alphau0(Everywhere)
  READS: GiRaFFE_HO::alpsqrtgam(Everywhere)
  READS: GiRaFFE_HO::AevolParen(Everywhere)
  READS: GiRaFFE_HO::PevolParenU0(Everywhere)
  READS: GiRaFFE_HO::PevolParenU1(Everywhere)
  READS: GiRaFFE_HO::PevolParenU2(Everywhere)
  READS: GiRaFFE_HO::gammaUU00(Everywhere)
  READS: GiRaFFE_HO::gammaUU01(Everywhere)
  READS: GiRaFFE_HO::gammaUU02(Everywhere)
  READS: GiRaFFE_HO::gammaUU11(Everywhere)
  READS: GiRaFFE_HO::gammaUU12(Everywhere)
  READS: GiRaFFE_HO::gammaUU22(Everywhere)
  READS: GiRaFFE_HO::gammadet(Everywhere)
  WRITES: GiRaFFE_HO::Stilde_rhsD0(Interior)
  WRITES: GiRaFFE_HO::Stilde_rhsD1(Interior)
  WRITES: GiRaFFE_HO::Stilde_rhsD2(Interior)
  WRITES: GiRaFFE_HO::A_rhsD0(Interior)
  WRITES: GiRaFFE_HO::A_rhsD1(Interior)
  WRITES: GiRaFFE_HO::A_rhsD2(Interior)
  WRITES: GiRaFFE_HO::psi6Phi_rhs(Interior)
} "Sets the GiRaFFE right-hand sides"

schedule GiRaFFE_HO_SelectBCs in MoL_PostStep
{
  LANG: C
  OPTIONS: level
  SYNC: GiRaFFE_vars
} "Boundaries of Maxwell's equations"

schedule GROUP ApplyBCs as GiRaFFE_HO_ApplyBCs in MoL_PostStep after GiRaFFE_HO_SelectBCs
{
} "Apply boundary conditions"


schedule GROUP ApplyBCs as GiRaFFE_HO_ApplyBCs at POSTRESTRICT
{
} "Apply boundary conditions"

schedule GiRaFFE_HO_RegisterVars in MoL_Register
{
  LANG: C
  OPTIONS: meta
} "Register Variables for MoL"

# Nontrivial primitives solve, for vx,vy,vz:
schedule GiRaFFE_conserv_to_prims_FFE in GiRaFFE_PostPostInitial after driver_A_to_B
{
  LANG: C
} "Applies the FFE condition B^2>E^2 and recomputes the velocities"

schedule HydroBase_to_GiRaFFE in GiRaFFE_PostPostInitial before driver_A_to_B
{
  LANG: C
  READS: HydroBase::Avec(Everywhere)
  READS: HydroBase::Bvec(Everywhere)
  READS: HydroBase::vel(Everywhere)
  WRITES: GiRaFFE_HO::BU0(Everywhere)
  WRITES: GiRaFFE_HO::BU1(Everywhere)
  WRITES: GiRaFFE_HO::BU2(Everywhere)
  WRITES: GiRaFFE_HO::AD0(Everywhere)
  WRITES: GiRaFFE_HO::AD1(Everywhere)
  WRITES: GiRaFFE_HO::AD2(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU0(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU1(Everywhere)
  WRITES: GiRaFFE_HO::ValenciavU2(Everywhere)
} "Converts the HydroBase variables to GiRaFFE variables"

schedule GiRaFFE_to_HydroBase AT CCTK_ANALYSIS AFTER ML_BSSN_evolCalcGroup
{
  LANG: C
  READS: GiRaFFE_HO::BU0(Everywhere)
  READS: GiRaFFE_HO::BU1(Everywhere)
  READS: GiRaFFE_HO::BU2(Everywhere)
  READS: GiRaFFE_HO::AD0(Everywhere)
  READS: GiRaFFE_HO::AD1(Everywhere)
  READS: GiRaFFE_HO::AD2(Everywhere)
  READS: GiRaFFE_HO::ValenciavU0(Everywhere)
  READS: GiRaFFE_HO::ValenciavU1(Everywhere)
  READS: GiRaFFE_HO::ValenciavU2(Everywhere)
  WRITES: HydroBase::Avec(Everywhere)
  WRITES: HydroBase::Bvec(Everywhere)
  WRITES: HydroBase::vel(Everywhere)
} "Converts the GiRaFFE variables to HydroBase variables"


Overwriting GiRaFFE_HO/schedule.ccl


#### Step 2c: Add the C file to Einstein Toolkit compilation list.

We will also need $\text{make.code.defn}$, which indicates the list of files that need to be compiled. This thorn only has the one C file to compile.

In [10]:
%%writefile GiRaFFE_HO/src/make.code.defn
SRCS = GiRaFFE.c driver_conserv_to_prims_FFE.C convert_ADM_to_BSSN__enforce_detgtij_eq_1__and_compute_gtupij.C \
       GiRaFFE_HydroBase_conversion.c

Overwriting GiRaFFE_HO/src/make.code.defn
